In [48]:
from bs4 import BeautifulSoup
import requests

page = requests.get("https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s?page=1")
soup = BeautifulSoup(page.content, 'html.parser')
soup

<!DOCTYPE html>

<html class="desktop withSiteHeaderTopFullImage">
<head>
<title>Best Books of the Decade: 2010s (7271 books)</title>
<meta content="7,271 books based on 13426 votes: The Fault in Our Stars by John Green, Divergent by Veronica Roth, Mockingjay by Suzanne Collins, Gone Girl by Gillian F..." name="description"/>
<meta content="telephone=no" name="format-detection"/>
<link href="https://www.goodreads.com/list/show/4093.Best_Books_of_the_Decade_2010s" rel="canonical"/>
<script type="text/javascript"> var ue_t0=window.ue_t0||+new Date();
 </script>
<script type="text/javascript">
    var ue_mid = "A1PQBFHBHS6YH1";
    var ue_sn = "www.goodreads.com";
    var ue_furl = "fls-na.amazon.com";
    var ue_sid = "327-0422613-1142712";
    var ue_id = "501HGAQ3VCGCHG0BG4TS";

    (function(e){var c=e;var a=c.ue||{};a.main_scope="mainscopecsm";a.q=[];a.t0=c.ue_t0||+new Date();a.d=g;function g(h){return +new Date()-(h?0:a.t0)}function d(h){return function(){a.q.push({n:h,a:arguments,t

In [91]:
books = soup.find_all('a', class_='bookTitle')

## need to go through multiple pages (5??)
# for i in range(5):
    # need to go to next page (more parsing)
    # need next for loop in this for loop

# for first page only
book_titles = []
book_descriptions = []
book_genres = []
idx = 0
# books
for i in books:
    link = "https://goodreads.com" + i['href']
    resp = requests.get(link)
    book_soup = BeautifulSoup(resp.content, 'html.parser')
    book_descriptions.append(book_soup.find("meta", property="og:description").get('content'))
    book_titles.append(book_soup.find("meta", property="og:title").get('content'))
   
    print(book_soup)
    #just to print first 5
    idx=idx+1
    if idx == 5:
        break
    
    # unsure abt this
    # title = book_soup.find('h1', id='bookTitle').text.strip()
    # genre = book_soup.find('a', class_='actionLinkLite bookPageGenreLink').text.strip()
    # summary = book_soup.find('div', id='description').find('span', class_='readable').text.strip()
    # get genre, title, description
book_titles



['The Fault in Our Stars',
 'Divergent (Divergent, #1)',
 'Mockingjay (The Hunger Games, #3)',
 'Gone Girl',
 'The Martian']

In [ ]:
# need to go to tbody html tag
# go to tr html tag
# go to td width="100%" html tag

# grab href to go to book page

# for each book:

    # go to class="Formatted"
    # store in book_descriptions
    
    # class="Text Text__title1"
    # store in book_titles
    
    # 5 times (for 5 genres)
    # go to class="BookPageMetadataSection__genreButton"
    
    
